In [ ]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.203.249:8501

[*********************100%%**********************]  1 of 1 completed
/Users/adarshpandey/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
[*********************100%%**********************]  1 of 1 completed
/Users/adarshpandey/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as data

In [ ]:
start = '2010-01-01'
end = '2021-12-31'


df = yf.download('AAPL', start=start, end=end)


today = datetime.date.today()
current_price_df = yf.download('AAPL', start=today, end=today)
current_price = current_price_df['Close'].iloc[-1]

df = df.append({'Open': current_price_df['Open'].iloc[-1],
                'High': current_price_df['High'].iloc[-1],
                'Low': current_price_df['Low'].iloc[-1],
                'Close': current_price,
                'Adj Close': current_price_df['Adj Close'].iloc[-1],
                'Volume': current_price_df['Volume'].iloc[-1]},
               ignore_index=True)


In [ ]:
df.tail()

In [ ]:
df = df.reset_index()
df.head()

In [ ]:
df = df.drop(['Date', 'Adj Close'], axis = 1)
df.head()
df['Current Price'] = df['Close']
plt.plot(df['Close'])

In [ ]:
plt.plot(df.Close)

In [ ]:
ma100 = df['Close'].rolling(100).mean()
ma200 = df['Close'].rolling(200).mean()
plt.figure(figsize = (12,6))
plt.plot(df.Close)
plt.plot(ma100, 'r')
plt.plot(ma200, 'g')
df.shape

In [ ]:
# Splitting data into training and testing
data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.70):int(len(df))])

print(data_training.shape)
print(data_testing.shape)
data_training.head()
data_testing.head()


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data_training_array = scaler.fit_transform(data_training)
data_training_array
x_train = []
y_train = []

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i-100:i])
    y_train.append(data_training_array[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
# ML Modelfrom 
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=60, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=80, activation='relu', return_sequences=True))
model.add(Dropout(0.4))

model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs = 50)

In [ ]:
model.save('keras_model.h5')

In [ ]:
data_testing.head()

In [ ]:
data_training.tail(100)

In [ ]:
import pandas as pd
!pip install --upgrade pandas

In [ ]:
past_100_days = pd.DataFrame(data_training.tail(100))
data_testing_df = pd.DataFrame(data_testing.values, columns=data_testing.columns)  # Ensure data_testing is a DataFrame

In [ ]:
final_df = pd.concat([past_100_days, data_testing_df], ignore_index=True)
final_df.head()

In [ ]:
input_data = scaler.fit_transform(final_df)
input_data.shape

In [ ]:
x_test = []
y_test = []

for i in range (100, input_data.shape[0]):
    x_test.append(input_data[i-100: i])
    y_test.append(input_data[i, 0])

x_test, y_test = np.array(x_test), np.array(y_test)
print(x_test.shape)
print(y_test.shape)

In [ ]:
y_predicted = model.predict(x_test)
y_predicted.shape

In [ ]:
scaler.scale_

In [ ]:
scale_factor = 1/0.00690691
y_predicted = y_predicted * scale_factor
y_test = y_test * scale_factor

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test, 'b', label = 'Original Price')
plt.plot(y_predicted, 'r', label = 'Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
# Calculate and print accuracy
threshold = 0.02 # Adjust as needed


# Calculate and print Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_predicted)
print(f'Mean Squared Error (MSE): {mse}')

# Calculate and print Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_predicted)
print(f'Mean Absolute Error (MAE): {mae}')

